## 向量嵌入

**假如说分词器是模型用来读取文本，那么向量嵌入(Enbedding)是模型用来理解文本的。**



我们可以很轻松的理解3维(x,y,z)，每一个维度可以当做是某方面的属性，其值是这个维度属性的值大小。   

现在假设我们有个768维，有($x_1$, $x_2$, $x_{...}$, $x_{768}$)共768个数。   
这意味着我们使用768个不同维度的数字属性来描述每个单词的重要部分。    
我们通过某种方式，把单词库中的各单词填充到这个768维的空间里面。每个单词有一个它自己的向量值。且其属性越接近的在这个向量空间中也越接近。

> 假设可以把一个词的**语义** 编码成描述其各种属性的数字集合。  
> 那么现在我们需要的是**一种将单词映射到它们的数字属性**的方法。   
> 维度很多，单词也多，我们靠人工执行这个任务特别特别的耗时，那么我们可以使用计算机来执行这个属性映射。

### 1. 直观感受BERT的向量嵌入

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import torch
from transformers import BertTokenizer, BertModel

### 1.1 分词

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
text = "I love python and transformer."

In [5]:
# 分词的时候，可以让结果直接返回为张量，传递参数return_tensors="pt"
# outputs = tokenizer([text], padding='max_length', max_length=15, return_tensors="pt")
outputs = tokenizer([text], padding='max_length', max_length=15)
outputs

{'input_ids': [[101, 1045, 2293, 18750, 1998, 10938, 2121, 1012, 102, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [6]:
tokenizer.decode([101, 1045, 2293, 18750, 102, 0, 0, 0])

'[CLS] i love python [SEP] [PAD] [PAD] [PAD]'

In [7]:
# 分词的时候如果传递了：return_tensors="pt"就不需要自己再把列表转换为张量了
input_ids = torch.tensor(outputs["input_ids"])
attention_mask = torch.tensor(outputs["attention_mask"])

### 2. 获取嵌入

In [8]:
model = BertModel.from_pretrained("bert-base-uncased")

In [9]:
results = model(input_ids, attention_mask=attention_mask)
type(results)

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [10]:
len(results)

2

In [11]:
# 查看返回的结果的keys
results.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [12]:
results['last_hidden_state'].shape

torch.Size([1, 15, 768])

In [13]:
results[0].shape

torch.Size([1, 15, 768])

In [14]:
# 我们再次查看一下输入的id列表的形状
input_ids.shape

torch.Size([1, 15])

In [15]:
results[0][0, 0, :10]

tensor([ 0.0589,  0.1161, -0.1256, -0.5529, -0.5943, -0.1909,  0.2947,  0.9077,
        -0.0863, -0.2452], grad_fn=<SliceBackward0>)